In [1]:
import pandas as pd
import re
data = pd.read_csv("FOIA_Request_Log_-_Ethics.csv")

In [2]:
#base code for this found here: http://stackoverflow.com/questions/25346058/removing-list-of-words-from-a-string
def remove_stopwords(string):
    stopwords = ['Candidate','Lobbyist']
    querywords = string.split()
    resultwords  = [word for word in querywords if word not in stopwords]
    return ' '.join(resultwords)

In [3]:
def pn_extractor(string):
    #rough way to remove stopwords for this purpose only
    try:
        string = remove_stopwords(string)
    except:
        pass
    initialdot_last_suffixdot = '([A-Z]\.\s[A-Z][a-z]{1,}\s[A-Z][a-z]\.)'
    first_initial_apostrophiclast = '([A-Z][a-z]{1,}\s[A-Z]\s[\w]{1,}\'[A-Z][a-z]{1,}\w)'
    first_initialdot_last = '([A-Z][a-z]{1,}\s[A-Z]\.\s[A-Z][a-z]{1,}\w)'
    first_last = '([A-Z][a-z]{1,}\s[A-Z][a-z]{1,}\w)'
    initialdot_last = '([A-Z]\.\s[A-Z][a-z]{1,}\w)'
    regexs = [initialdot_last_suffixdot, first_initial_apostrophiclast, first_initialdot_last, first_last, initialdot_last]
    proper_names = []
    try:
        for regex in regexs:
            pattern = re.compile(regex)
            pn_list = re.findall(pattern, string)
            for pn in pn_list:
                proper_names.append(pn)
    except:
        proper_names.append("None")
    return proper_names

In [4]:
data["FINANCIAL INTERESTS Extracted People"] = data["DESCRIPTION OF REQUEST - STATEMENTS OF FINANCIAL INTERESTS"].apply(pn_extractor)

In [5]:
data["LOBBYISTS FILINGS Extracted People"] = data["DESCRIPTION OF REQUEST - LOBBYISTS FILINGS"].apply(pn_extractor)

In [6]:
data["OTHER Extracted People"] = data["DESCRIPTION OF REQUEST - OTHER"].apply(pn_extractor)

In [7]:
data.head(20)

,REQUESTOR NAME,ORGANIZATION,DATE RECEIVED,DUE DATE,DESCRIPTION OF REQUEST - STATEMENTS OF FINANCIAL INTERESTS,DESCRIPTION OF REQUEST - LOBBYISTS FILINGS,DESCRIPTION OF REQUEST - OTHER,FINANCIAL INTERESTS Extracted People,LOBBYISTS FILINGS Extracted People,OTHER Extracted People
0,A. Jones,Illinois Gaming Board,05/07/2010,05/14/2010,NaN,J. Montgomery Sr. 2003-10 available,NaN,[None],"[J. Montgomery Sr., J. Montgomery]",[None]
1,J. Chase,Tribune,05/12/2010,05/19/2010,Dennis Redmond all forms; all available,NaN,NaN,[Dennis Redmond],[None],[None]
2,Ian Main,Carpenters Local Union #58,05/18/2010,05/25/2010,Richard Rodriguez 2006-09 all available; John ...,NaN,NaN,"[Richard Rodriguez, John Vasilj]",[None],[None]
3,Merema,NaN,06/01/2010,06/08/2010,NaN,"Laube 2009-2010 all were available, and provided",NaN,[None],[],[None]
4,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None]
5,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None]
6,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None]
7,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None]
8,J. Sabel,ETS Environmental,06/07/2010,06/14/2010,NaN,NaN,environmental records re Faith Tabernacle Church,[None],[None],[Faith Tabernacle]
9,T. Novak,Sun Times,06/08/2010,06/15/2010,NaN,"E. Kruse 2007-9 all available, produced re Str...",NaN,[None],"[Structured Dev, E. Kruse]",[None]


In [8]:
#errors out on names with ' in them like O'Gorman
#solved for now by just replacing with nothing, but not final solution
journalist_nodes = []
journalist_short_names = []
for journalist in data["REQUESTOR NAME"]:
    try:
        journalist = journalist.replace("'", "")
        short_name = journalist.replace(" ", "")
        short_name = short_name.replace(".", "")
        short_name = short_name.replace("-","")
        query = "CREATE (%s: Journalist {name: '%s'})" % (short_name, journalist)
        journalist_nodes.append(query)
        journalist_short_names.append(short_name)
    except:
        journalist_short_names.append("None")
journalist_nodes[0:5]

["CREATE (AJones: Journalist {name: 'A. Jones'})",
 "CREATE (JChase: Journalist {name: 'J. Chase'})",
 "CREATE (IanMain: Journalist {name: 'Ian Main'})",
 "CREATE (Merema: Journalist {name: 'Merema'})",
 "CREATE (TNovak: Journalist {name: 'T. Novak'})"]

In [9]:
#also a problem with duplicate short names
#just copying in groups right now to avoid repeated JStone
#but obviously I can write code to add "2", "3" etc. to the end of duplicates
for query in set(journalist_nodes):
    print(query)

CREATE (DBernstein: Journalist {name: 'D. Bernstein'})
CREATE (TFurman: Journalist {name: 'T. Furman'})
CREATE (JKibbey: Journalist {name: 'J. Kibbey'})
CREATE (BJLutz: Journalist {name: 'BJ Lutz'})
CREATE (MScarpace: Journalist {name: 'M. Scarpace'})
CREATE (EMates: Journalist {name: 'E. Mates'})
CREATE (WDonoghue: Journalist {name: 'W. Donoghue'})
CREATE (JCombs: Journalist {name: 'J. Combs'})
CREATE (AdamAndrzejewski: Journalist {name: 'Adam Andrzejewski'})
CREATE (Merema: Journalist {name: 'Merema'})
CREATE (USPaysforZionistorientedpolicies: Journalist {name: 'US Pays for Zionist-oriented policies'})
CREATE (PMerrion: Journalist {name: 'P. Merrion'})
CREATE (LNickus: Journalist {name: 'L. Nickus'})
CREATE (CToner: Journalist {name: 'C. Toner'})
CREATE (CMitchell: Journalist {name: 'C. Mitchell'})
CREATE (YLaseca: Journalist {name: 'Y. Laseca'})
CREATE (JWise: Journalist {name: 'J. Wise'})
CREATE (JHernandez: Journalist {name: 'J. Hernandez'})
CREATE (TBowen: Journalist {name: 'T. B

In [10]:
journalist_merges = []
for journ in data["REQUESTOR NAME"]:
    try:
        journ = journ.replace("'", "")
        query = "MERGE (n:Journalist { name:'%s' }), " % journ
        journalist_merges.append(query)
    except:
        pass
journalist_merges[0:5]

["MERGE (n:Journalist { name:'A. Jones' }), ",
 "MERGE (n:Journalist { name:'J. Chase' }), ",
 "MERGE (n:Journalist { name:'Ian Main' }), ",
 "MERGE (n:Journalist { name:'Merema' }), ",
 "MERGE (n:Journalist { name:'T. Novak' }), "]

In [11]:
#MERGE (n:Person { name: 'Ann' }) RETURN n
for query in set(journalist_merges):
    print(query)

MERGE (n:Journalist { name:'Linda Butterfield' }), 
MERGE (n:Journalist { name:'R. Chisessi' }), 
MERGE (n:Journalist { name:'A. Kirsch' }), 
MERGE (n:Journalist { name:'T. Nazanin' }), 
MERGE (n:Journalist { name:'S. and M. Sirot' }), 
MERGE (n:Journalist { name:'J. Ailey' }), 
MERGE (n:Journalist { name:'B. Gryll' }), 
MERGE (n:Journalist { name:'J. Dispensa' }), 
MERGE (n:Journalist { name:'R. Quijano' }), 
MERGE (n:Journalist { name:'D. Kidwell' }), 
MERGE (n:Journalist { name:'M. Dwyer' }), 
MERGE (n:Journalist { name:'E. Sandoval' }), 
MERGE (n:Journalist { name:'R. Drew' }), 
MERGE (n:Journalist { name:'P. Merrion' }), 
MERGE (n:Journalist { name:'T. Wilkins' }), 
MERGE (n:Journalist { name:'J. Howerton' }), 
MERGE (n:Journalist { name:'H. Alex' }), 
MERGE (n:Journalist { name:'S. Handy' }), 
MERGE (n:Journalist { name:'David Tanner' }), 
MERGE (n:Journalist { name:'N. Lange' }), 
MERGE (n:Journalist { name:'L. Ketzeback' }), 
MERGE (n:Journalist { name:'A. Finko' }), 
MERGE (n:

In [12]:
data["Journalist Short Names"] = journalist_short_names
data.head()

,REQUESTOR NAME,ORGANIZATION,DATE RECEIVED,DUE DATE,DESCRIPTION OF REQUEST - STATEMENTS OF FINANCIAL INTERESTS,DESCRIPTION OF REQUEST - LOBBYISTS FILINGS,DESCRIPTION OF REQUEST - OTHER,FINANCIAL INTERESTS Extracted People,LOBBYISTS FILINGS Extracted People,OTHER Extracted People,Journalist Short Names
0,A. Jones,Illinois Gaming Board,05/07/2010,05/14/2010,NaN,J. Montgomery Sr. 2003-10 available,NaN,[None],"[J. Montgomery Sr., J. Montgomery]",[None],AJones
1,J. Chase,Tribune,05/12/2010,05/19/2010,Dennis Redmond all forms; all available,NaN,NaN,[Dennis Redmond],[None],[None],JChase
2,Ian Main,Carpenters Local Union #58,05/18/2010,05/25/2010,Richard Rodriguez 2006-09 all available; John ...,NaN,NaN,"[Richard Rodriguez, John Vasilj]",[None],[None],IanMain
3,Merema,NaN,06/01/2010,06/08/2010,NaN,"Laube 2009-2010 all were available, and provided",NaN,[None],[],[None],Merema
4,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None],TNovak


In [13]:
organization_nodes = []
org_short_names = []
for org in data["ORGANIZATION"]:
    try:
        short_name = ""
        org = org.replace("'", "")
        short_name = org.replace(" ", "")
        short_name = short_name.replace(".", "")
        short_name = short_name.replace("-","")
        short_name = short_name.replace("#", "")
        short_name = short_name.replace("&", "")
        short_name = short_name.replace("@", "")
        short_name = short_name.replace(",", "")
        short_name = short_name.replace("/", "")
        short_name = short_name.replace("[", "")
        short_name = short_name.replace("]", "")
        query = "CREATE (%s: Organization {name: '%s'})" % (short_name, org)
        organization_nodes.append(query)
        org_short_names.append(short_name)
    except:
        org_short_names.append("None")
organization_nodes[0:5]

["CREATE (IllinoisGamingBoard: Organization {name: 'Illinois Gaming Board'})",
 "CREATE (Tribune: Organization {name: 'Tribune'})",
 "CREATE (CarpentersLocalUnion58: Organization {name: 'Carpenters Local Union #58'})",
 "CREATE (SunTimes: Organization {name: 'Sun Times'})",
 "CREATE (SunTimes: Organization {name: 'Sun Times'})"]

In [14]:
for query in set(organization_nodes):
    print(query)

CREATE (IITChicagoKentCollegeofLaw: Organization {name: 'IIT Chicago-Kent College of Law'})
CREATE (ChicagoMagazine: Organization {name: 'Chicago Magazine'})
CREATE (CharlesRGryllLtd: Organization {name: 'Charles R. Gryll, Ltd.'})
CREATE (AmericaRising: Organization {name: 'America Rising'})
CREATE (SubstanceNews: Organization {name: 'Substance News'})
CREATE (NorthwesternUniversity: Organization {name: 'Northwestern University'})
CREATE (CCEG: Organization {name: 'CCEG'})
CREATE (SamuelsCoInc: Organization {name: 'Samuels & Co., Inc.'})
CREATE (newchicagollc: Organization {name: 'new chicago llc'})
CREATE (ProjectSix: Organization {name: 'Project Six'})
CREATE (EarlyandOftenTheChicagoNewCooperative: Organization {name: 'Early and Often/The Chicago New Cooperative'})
CREATE (MedillSchoolofJournalism: Organization {name: 'Medill School of Journalism'})
CREATE (Aramark: Organization {name: 'Aramark'})
CREATE (WBEZ: Organization {name: 'WBEZ'})
CREATE (SunTimes: Organization {name: 'Sun-T

In [15]:
data["Organization Short Names"] = org_short_names
data.head()

,REQUESTOR NAME,ORGANIZATION,DATE RECEIVED,DUE DATE,DESCRIPTION OF REQUEST - STATEMENTS OF FINANCIAL INTERESTS,DESCRIPTION OF REQUEST - LOBBYISTS FILINGS,DESCRIPTION OF REQUEST - OTHER,FINANCIAL INTERESTS Extracted People,LOBBYISTS FILINGS Extracted People,OTHER Extracted People,Journalist Short Names,Organization Short Names
0,A. Jones,Illinois Gaming Board,05/07/2010,05/14/2010,NaN,J. Montgomery Sr. 2003-10 available,NaN,[None],"[J. Montgomery Sr., J. Montgomery]",[None],AJones,IllinoisGamingBoard
1,J. Chase,Tribune,05/12/2010,05/19/2010,Dennis Redmond all forms; all available,NaN,NaN,[Dennis Redmond],[None],[None],JChase,Tribune
2,Ian Main,Carpenters Local Union #58,05/18/2010,05/25/2010,Richard Rodriguez 2006-09 all available; John ...,NaN,NaN,"[Richard Rodriguez, John Vasilj]",[None],[None],IanMain,CarpentersLocalUnion58
3,Merema,NaN,06/01/2010,06/08/2010,NaN,"Laube 2009-2010 all were available, and provided",NaN,[None],[],[None],Merema,None
4,T. Novak,Sun Times,06/04/2010,06/11/2010,NaN,All lobbyists for Higgins Development 6 files ...,NaN,[None],"[Higgins Development, Fred Latsko, Structure M...",[None],TNovak,SunTimes


In [16]:
journalist_relationships = []
for journ, org in zip(data["Journalist Short Names"], data["Organization Short Names"]):
    if org == "None":
        pass
    else:
        query = "CREATE (%s)-[:WORKS_FOR]->(%s)" % (journ, org)
        journalist_relationships.append(query)
journalist_relationships[0:5]

['CREATE (AJones)-[:WORKS_FOR]->(IllinoisGamingBoard)',
 'CREATE (JChase)-[:WORKS_FOR]->(Tribune)',
 'CREATE (IanMain)-[:WORKS_FOR]->(CarpentersLocalUnion58)',
 'CREATE (TNovak)-[:WORKS_FOR]->(SunTimes)',
 'CREATE (TNovak)-[:WORKS_FOR]->(SunTimes)']

In [17]:
for query in set(journalist_relationships):
    print(query)

CREATE (PMerrion)-[:WORKS_FOR]->(Crains)
CREATE (MWalton)-[:WORKS_FOR]->(MedillWatchdogIniativeNorthwestern)
CREATE (JMcCall)-[:WORKS_FOR]->(NationalCorporateResearchLtd)
CREATE (EBalderas)-[:WORKS_FOR]->(IllinoisSenateDemocrats)
CREATE (SOstman)-[:WORKS_FOR]->(AustinTalksorg)
CREATE (HDardick)-[:WORKS_FOR]->(Tribune)
CREATE (TBowen)-[:WORKS_FOR]->(newchicagollc)
CREATE (None)-[:WORKS_FOR]->(SunTimes)
CREATE (CMertens)-[:WORKS_FOR]->(DePaulUniversity)
CREATE (JByrnes)-[:WORKS_FOR]->(Tribune)
CREATE (JWalsh)-[:WORKS_FOR]->(MedillWatchdogIniativeNorthwestern)
CREATE (JordanFein)-[:WORKS_FOR]->(Unitehere)
CREATE (CToner)-[:WORKS_FOR]->(BetterGovernmentAssociation)
CREATE (AEmmanuel)-[:WORKS_FOR]->(DNAinfocomChicago)
CREATE (AFinko)-[:WORKS_FOR]->(AndrewFinkoPC)
CREATE (ASwartz)-[:WORKS_FOR]->(MacabeeGroup)
CREATE (SClear)-[:WORKS_FOR]->(IHCFinancialGroupInc)
CREATE (PMcDonough)-[:WORKS_FOR]->(MunicipalAdvocateGeneralParkRidgeIL)
CREATE (AJones)-[:WORKS_FOR]->(IllinoisGamingBoard)
CREATE (

In [18]:
org_relationships = []
for journ, org in zip(data["Journalist Short Names"], data["Organization Short Names"]):
    if org == "None":
        pass
    else:
        query = "CREATE (%s)-[:EMPLOYS]->(%s)" % (org, journ)
        org_relationships.append(query)
org_relationships[0:5] 

['CREATE (IllinoisGamingBoard)-[:EMPLOYS]->(AJones)',
 'CREATE (Tribune)-[:EMPLOYS]->(JChase)',
 'CREATE (CarpentersLocalUnion58)-[:EMPLOYS]->(IanMain)',
 'CREATE (SunTimes)-[:EMPLOYS]->(TNovak)',
 'CREATE (SunTimes)-[:EMPLOYS]->(TNovak)']

In [19]:
for query in set(org_relationships):
    print(query)

CREATE (CCEG)-[:EMPLOYS]->(ACastillo)
CREATE (NorthwesternUniversity)-[:EMPLOYS]->(MWalton)
CREATE (MarkJKupiecAssociatesAttorneysatLaw)-[:EMPLOYS]->(MKupiec)
CREATE (DepresSchwartzGeogheganLtd)-[:EMPLOYS]->(TGeoghegan)
CREATE (MacabeeGroup)-[:EMPLOYS]->(ASwartz)
CREATE (SunTimes)-[:EMPLOYS]->(FSpielman)
CREATE (Tribune)-[:EMPLOYS]->(JChase)
CREATE (ChicagoNewsCooperative)-[:EMPLOYS]->(HClauss)
CREATE (SunTimes)-[:EMPLOYS]->(CFusco)
CREATE (SubstanceNews)-[:EMPLOYS]->(JKugler)
CREATE (EarlyandOftenTheChicagoNewsCooperative)-[:EMPLOYS]->(JDispensa)
CREATE (Tribune)-[:EMPLOYS]->(JHopkins)
CREATE (Tribune)-[:EMPLOYS]->(DKidwell)
CREATE (SuperiorCorpofIL)-[:EMPLOYS]->(MLivesay)
CREATE (CityofChicago)-[:EMPLOYS]->(TWilkins)
CREATE (LoevyLoevy)-[:EMPLOYS]->(MTopic)
CREATE (IHCFinancialGroupInc)-[:EMPLOYS]->(SClear)
CREATE (ThomsonReuters)-[:EMPLOYS]->(CDuncan)
CREATE (CityofChicago)-[:EMPLOYS]->(SSheely)
CREATE (ProjectSix)-[:EMPLOYS]->(FKhan)
CREATE (ilivelife)-[:EMPLOYS]->(LKetzeback)
CREA

In [20]:
foiaed_nodes_1 = []
foiaed_short_names_1 = []
for foiaed in data["FINANCIAL INTERESTS Extracted People"]:
        list_short_names = []
        for person in foiaed:
            if person == "None":
                list_short_names.append("None")
            else:
                person = person.replace("'", "")
                short_name = person.replace(" ", "")
                short_name = short_name.replace(".", "")
                short_name = short_name.replace("-","")
                query = "CREATE (%s: Investigatee {name: '%s'})" % (short_name, person)
                foiaed_nodes_1.append(query)
                list_short_names.append(short_name)   
        foiaed_short_names_1.append(list_short_names)
foiaed_nodes_1[0:5]

["CREATE (DennisRedmond: Investigatee {name: 'Dennis Redmond'})",
 "CREATE (RichardRodriguez: Investigatee {name: 'Richard Rodriguez'})",
 "CREATE (JohnVasilj: Investigatee {name: 'John Vasilj'})",
 "CREATE (CityColleges: Investigatee {name: 'City Colleges'})",
 "CREATE (RahmEmanuel: Investigatee {name: 'Rahm Emanuel'})"]

In [21]:
foiaed_short_names_1[0:5]

[['None'],
 ['DennisRedmond'],
 ['RichardRodriguez', 'JohnVasilj'],
 ['None'],
 ['None']]

In [22]:
for query in set(foiaed_nodes_1):
    print(query)

CREATE (JohnMolloy: Investigatee {name: 'John Molloy'})
CREATE (EdBurke: Investigatee {name: 'Ed Burke'})
CREATE (JohnVasilj: Investigatee {name: 'John Vasilj'})
CREATE (MNewman: Investigatee {name: 'M. Newman'})
CREATE (JRodriguez: Investigatee {name: 'J. Rodriguez'})
CREATE (MHernandez: Investigatee {name: 'M. Hernandez'})
CREATE (JMeeks: Investigatee {name: 'J. Meeks'})
CREATE (RCowen: Investigatee {name: 'R. Cowen'})
CREATE (PatricialMolloy: Investigatee {name: 'Patricial Molloy'})
CREATE (BMatos: Investigatee {name: 'B. Matos'})
CREATE (JamesTOShea: Investigatee {name: 'James T OShea'})
CREATE (MargaretMolly: Investigatee {name: 'Margaret Molly'})
CREATE (SMendoza: Investigatee {name: 'S. Mendoza'})
CREATE (BrendanReilly: Investigatee {name: 'Brendan Reilly'})
CREATE (MichelleMMurphy: Investigatee {name: 'Michelle M. Murphy'})
CREATE (RichardRybicki: Investigatee {name: 'Richard Rybicki'})
CREATE (RobertoMontano: Investigatee {name: 'Roberto Montano'})
CREATE (WSiegmund: Investiga

In [23]:
data["Foiaed 1 Short Names"] = foiaed_short_names_1

In [24]:
journ_foiaed = []
for journ, foiaed in zip(data["Journalist Short Names"], data["Foiaed 1 Short Names"]):
    if foiaed == "None":
        pass
    else:
        for person in foiaed:
            if person == "None":
                pass
            else:
                query = "CREATE (%s)-[:FOIAED]->(%s)" % (journ, person)
                journ_foiaed.append(query)
journ_foiaed[0:5]

['CREATE (JChase)-[:FOIAED]->(DennisRedmond)',
 'CREATE (IanMain)-[:FOIAED]->(RichardRodriguez)',
 'CREATE (IanMain)-[:FOIAED]->(JohnVasilj)',
 'CREATE (SCrets)-[:FOIAED]->(CityColleges)',
 'CREATE (JWise)-[:FOIAED]->(RahmEmanuel)']

In [25]:
for query in set(journ_foiaed):
    print(query)

CREATE (JStone)-[:FOIAED]->(RCowen)
CREATE (JDispensa)-[:FOIAED]->(DelValle)
CREATE (JDispensa)-[:FOIAED]->(RGonzalez)
CREATE (HValkavich)-[:FOIAED]->(JoseSantiago)
CREATE (TNovak)-[:FOIAED]->(MichelleMMurphy)
CREATE (EReyes)-[:FOIAED]->(AldRoberto)
CREATE (MWahadlo)-[:FOIAED]->(CheRhymefest)
CREATE (IanMain)-[:FOIAED]->(JohnVasilj)
CREATE (JDispensa)-[:FOIAED]->(GBrewer)
CREATE (JDispensa)-[:FOIAED]->(JHeidt)
CREATE (JAiley)-[:FOIAED]->(JIniguez)
CREATE (JDispensa)-[:FOIAED]->(TMotzny)
CREATE (JAiley)-[:FOIAED]->(AMercado)
CREATE (JDispensa)-[:FOIAED]->(AKlocek)
CREATE (JDispensa)-[:FOIAED]->(SMendoza)
CREATE (HValkavich)-[:FOIAED]->(JohnBrooks)
CREATE (JDispensa)-[:FOIAED]->(PFoster)
CREATE (JDispensa)-[:FOIAED]->(HOsterman)
CREATE (JDispensa)-[:FOIAED]->(GRobinson)
CREATE (AArkin)-[:FOIAED]->(EdwardMBurke)
CREATE (JDispensa)-[:FOIAED]->(ABocanegra)
CREATE (TNovak)-[:FOIAED]->(RichardRybicki)
CREATE (RSattler)-[:FOIAED]->(TomGCaravette)
CREATE (TNovak)-[:FOIAED]->(MMolloy)
CREATE (AK

In [26]:
foiaed_by = []
for journ, foiaed in zip(data["Journalist Short Names"], data["Foiaed 1 Short Names"]):
    if foiaed == "None":
        pass
    else:
        for person in foiaed:
            if person == "None":
                pass
            else:
                query = "CREATE (%s)-[:INVESTIGATED_BY]->(%s)" % (person, journ)
                foiaed_by.append(query)
foiaed_by[0:5]

['CREATE (DennisRedmond)-[:INVESTIGATED_BY]->(JChase)',
 'CREATE (RichardRodriguez)-[:INVESTIGATED_BY]->(IanMain)',
 'CREATE (JohnVasilj)-[:INVESTIGATED_BY]->(IanMain)',
 'CREATE (CityColleges)-[:INVESTIGATED_BY]->(SCrets)',
 'CREATE (RahmEmanuel)-[:INVESTIGATED_BY]->(JWise)']

In [27]:
for query in set(foiaed_by):
    print(query)

CREATE (AMercado)-[:INVESTIGATED_BY]->(JAiley)
CREATE (BdElec)-[:INVESTIGATED_BY]->(JStone)
CREATE (MDel)-[:INVESTIGATED_BY]->(JDispensa)
CREATE (AllAlderman)-[:INVESTIGATED_BY]->(FNirappil)
CREATE (WalterBurnett)-[:INVESTIGATED_BY]->(GBeukinga)
CREATE (JIniguez)-[:INVESTIGATED_BY]->(JAiley)
CREATE (TomTunney)-[:INVESTIGATED_BY]->(PatrickRehkamp)
CREATE (JGuereca)-[:INVESTIGATED_BY]->(JAiley)
CREATE (JBush)-[:INVESTIGATED_BY]->(JDispensa)
CREATE (CityColleges)-[:INVESTIGATED_BY]->(MScarpace)
CREATE (RaymondOrozco)-[:INVESTIGATED_BY]->(HValkavich)
CREATE (GCardenas)-[:INVESTIGATED_BY]->(ABocanegra)
CREATE (JGarrido)-[:INVESTIGATED_BY]->(JDispensa)
CREATE (MNewman)-[:INVESTIGATED_BY]->(JDispensa)
CREATE (BrockMerck)-[:INVESTIGATED_BY]->(JHernandez)
CREATE (MBurke)-[:INVESTIGATED_BY]->(AArkin)
CREATE (MichelleMMurphy)-[:INVESTIGATED_BY]->(TNovak)
CREATE (BRoberts)-[:INVESTIGATED_BY]->(JDispensa)
CREATE (JenniferAKramer)-[:INVESTIGATED_BY]->(RLuther)
CREATE (TThomas)-[:INVESTIGATED_BY]->(H

In [37]:
#t	MATCH (a:Investigatee { name: 'Gery Chico' }), (b:Journalist { name: 'A. Castillo' }) MERGE (a)-[:INVESTIGATED_BY]->(b)
#MATCH (a:Person { name: 'Ann' }), (b:Person { name: 'Dan' })
#MERGE (a)-[:KNOWS]->(b)

foiaed_merges_1 = []
for foiaed, journ in zip(data["FINANCIAL INTERESTS Extracted People"], data["REQUESTOR NAME"]):
        for person in foiaed:
            if person == "None":
                pass
            else:
                person = person.replace("'", "")
                journ = journ.replace("'", "")
                query = "MATCH (a:Investigatee { name: '%s' }), (b:Journalist { name: '%s' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)" % (person, journ)
                foiaed_merges_1.append(query)
    
foiaed_merges_1[0:5]

["MATCH (a:Investigatee { name: 'Dennis Redmond' }), (b:Journalist { name: 'J. Chase' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)",
 "MATCH (a:Investigatee { name: 'Richard Rodriguez' }), (b:Journalist { name: 'Ian Main' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)",
 "MATCH (a:Investigatee { name: 'John Vasilj' }), (b:Journalist { name: 'Ian Main' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)",
 "MATCH (a:Investigatee { name: 'City Colleges' }), (b:Journalist { name: 'S. Crets' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)",
 "MATCH (a:Investigatee { name: 'Rahm Emanuel' }), (b:Journalist { name: 'J. Wise' })\nMERGE (a)-[:INVESTIGATED_BY]->(b)"]

In [38]:
#okay missing something to do this repeatedly, as this works for each match/merge pair
#but it won't let me batch them
#so for now, just pasting in each one just to get this done since small enough
for query in set(foiaed_merges_1):
    print(query)

MATCH (a:Investigatee { name: 'M. Smith' }), (b:Journalist { name: 'J. Dispensa' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'Walter Burnett' }), (b:Journalist { name: 'Linda Butterfield' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'S. Mendoza' }), (b:Journalist { name: 'J. Dispensa' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'W. Siegmund' }), (b:Journalist { name: 'J. Dispensa' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'William D. Burns' }), (b:Journalist { name: 'F. Nirappil' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'D. Burns' }), (b:Journalist { name: 'F. Nirappil' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'Jose Santiago' }), (b:Journalist { name: 'H. Valkavich' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee { name: 'Thomas Hughes' }), (b:Journalist { name: 'T. Novak' })
MERGE (a)-[:INVESTIGATED_BY]->(b)
MATCH (a:Investigatee {